In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='agol_profile', verify_cert=False) 

## Working with Users

<center><img src="./img/usermanagement.jpg" /></center>

- The GIS object contains functionality to create, manage, and monitor users

## 10.6.x vs 10.7/10.8 Portal

- At **10.7** the user account types changed!  
- Levels essentially are deprecated and the concept of User Types were introduced
- Provides more flexibility in what a user can do.
- Python API for ArcGIS supports **BOTH** ways.

## Simple User Reports: `counts()` 

- simple report on the number of licenses currently used

In [ ]:
df = gis.users.counts(type="user_type")
df

In [ ]:
gis.users.counts(type='roles')

## User Counts

In [ ]:
df = gis.users.counts(type="user_type")
df['count'].sum()

## Creating New Users

- `gis.users.create()` provides a seamless operation to generate new users
- Support Pre/Post 10.7 user generation 
    + one function for both platforms

**Example: Create a Level 1 Compatible User**

In [ ]:
user = gis.users.create(username="AilDAadsf",
                password='Sch0II4r0ckslap',
                firstname="General",
                lastname="Account",
                email="gac@esri.com",
                role="iAAAAAAAAAAAAAAA",
                user_type="viewerUT")
user

In [ ]:
user.delete()

## Example: Creating a Content Creator

In [ ]:
gis.users.search("AilDAadsf")

In [ ]:
user = gis.users.create(username="AilDAadsf",
                password='Sch0II4r0ckslap',
                firstname="General",
                lastname="Account",
                email="gac@esri.com",
                user_type='creatorUT')
user

In [ ]:
user.reset(new_security_question=1,
           new_security_answer="Redlands",
           password='Sch0II4r0ckslap',
           new_password="IMgoing2LA")

In [ ]:
gis_new = GIS(url=gis._url, 
              username='AilDAadsf', password="IMgoing2LA", 
              set_active=False)
gis_new.users.me

## Delete the User

In [ ]:
for l in gis.admin.license.all():
    entitle = l.user_entitlement(username="AilDAadsf")
    if 'entitlements' in entitle:
        print(entitle['entitlements'])
        l.revoke(username="AilDAadsf", entitlements=entitle['entitlements'], suppress_email=True)
gis.users.get("AilDAadsf").delete()

## Updating Users

- Like `Item` classes, users have `update()`


In [ ]:
user = gis.users.create(username="bruce.account",
                password='dnc1ng1ntheDARK',
                firstname="Unknown",
                lastname="Account",
                email="gac@esri.com",
                role="iAAAAAAAAAAAAAAA",
                user_type="viewerUT")
user

In [ ]:
user.update(first_name="Bruce", last_name="Springsteen", 
            thumbnail=r"./data/user_thumbnail.jpg")
user

## **Question: How do we let Bruce create content?**

In [ ]:
print((user.userLicenseTypeId, user.role))
user.update_license_type(user_type="creatorUT")
user.update_role("org_publisher")
print((user.userLicenseTypeId, user.role))

## **Recall: How do you add content to a user?**

In [ ]:
fp = r"./data/rivers.gdb.zip"
item_properties = {
    'title' : "US Rivers",
    'type' : 'File Geodatabase',
    'tags' : ['Rivers']
}
river_item = gis.content.add(data=fp, item_properties=item_properties, owner=user)

In [ ]:
p_river = river_item.publish()
p_river

### Accessing User Content

### Searching for Users

- `search` provides the ability to find users
- Locates both system and user accounts

In [ ]:
users = gis.users.search("bruce*")
users

## List a User's Content

In [ ]:
items = users[0].items()
items

## Reassigning Content

- Assign content from one user to another

In [ ]:
for item in users[0].items():
    if item.type == "Feature Service":
        print(item.reassign_to(target_owner=gis.users.me.username))
users[0].items()

## Removing Users

- Before deleting, users must be free of:

    + Items
    + Groups
    + Licensing 

In [ ]:
users[0].delete()

In [ ]:
gis.users.search("bruce")

## Group Manager

### The Group

<img src="./img/groups.jpeg" />

- `GroupManager` provides tools to manage groups
    + create
    + update
    + delete
    + management content

In [ ]:
type(gis.groups)

### Searching for Groups

In [ ]:
agol_gis = GIS(profile='agol_profile')
agol_gis.groups.search(query="Complianc*")

### Managing Groups

#### Creating a Group

In [ ]:
group = gis.groups.create(title="AprilDemoGroup",
                  tags=['demo', 'eraseme'], 
                  is_invitation_only=False,
                  auto_join=True)
group

In [ ]:
group.update(thumbnail=r"./data/group-icon.png")
group

#### Assigning Items to Group

In [ ]:
item = gis.users.me.items()[0]
item.share(groups=[group])
item.shared_with

#### Adding Users to Group

In [ ]:
group.add_users(usernames=[gis.users.search("*")])

#### Deleting a Group

In [ ]:
group.delete()